In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import seaborn as sns
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings

# skew and kurtosis
from scipy.stats import skew, kurtosis

import os
import sys
from dotenv import load_dotenv

REPO_PATH = os.getenv('REPO_PATH')

sys.path.insert(0, rf'{REPO_PATH}src_HF')
from utils.main_utils import load_json
from utils.ts_utils import FuturesBank

warnings.filterwarnings("ignore")
plt.style.use('science')

In [ ]:
CONFIG = load_json(rf'{REPO_PATH}src_HF\plot_config.json')

df_LCOc1 = pd.read_csv(rf'{REPO_PATH}data\time_series\LCOc1_High_Frequency.csv') # Brent Crud Oil
df_CLc1 = pd.read_csv(rf'{REPO_PATH}data\time_series\CLc1_High_Frequency.csv') # WTI Crud Oil

# create datetime index
df_LCOc1.index = pd.to_datetime(df_LCOc1['Date'])
df_CLc1.index = pd.to_datetime(df_CLc1['Date'])

colormap = plt.cm.get_cmap('twilight', 256)
palette = [colormap(i) for i in (70, 180)]

### Price plot

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
palette = [colormap(i) for i in (70, 180)]
df_LCOc1['CLOSE'].plot(label='ICE Brent Crude (LCOc1)', ax=ax, lw=0.8, color=palette[0])
df_CLc1['CLOSE'].plot(label='NYMEX WTI Crude (CLc1)', ax=ax, lw=0.8, color=palette[1])
ax.set_ylabel('Price (USD)', fontsize=14)
ax.set_xlabel('Date', fontsize=14)
ax.legend(frameon=False, loc='upper left', fontsize=13)
ax.tick_params(axis='x', labelsize=14)
ax.grid(alpha=0.2)

fig.savefig('images/crude_oil_price.png', bbox_inches='tight', dpi=200)

### Log returns analysis

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10,5), dpi=200, sharex=True)

labels = ['ICE Brent Crude (LCOc1)', 'NYMEX WTI Crude (CLc1)']

for i, df in enumerate([df_LCOc1, df_CLc1]):
    logret = np.log(df['CLOSE']).diff()
    logret_resampled = logret.resample('1D').sum() * 100
    # descriptive statistics
    display(logret_resampled.describe())
    skewness = skew(logret_resampled.dropna())
    kurt = kurtosis(logret_resampled.dropna())
    print(f'Skewness: {skewness:.2f}, Kurtosis: {kurt:.2f}')

    axs[i].plot(logret * 100, lw=0.8, color=palette[i], label=labels[i])
    axs[i].set_ylabel('Log Return (\%)', fontsize=16)
    axs[i].legend(frameon=False, loc='upper left', fontsize=15)

axs[-1].set_xlabel('Date', fontsize=16)
fig.tight_layout(pad=-0.5)

fig.savefig('images/crude_oil_log_returns.png', bbox_inches='tight')

### Volume plot

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
# create a stacked bar plot resampled bby yearly frequency
volume_df = pd.concat(
    [
        df_LCOc1['VOLUME'].resample('w').sum(), 
        df_CLc1['VOLUME'].resample('w').sum()
    ], axis=1
)
volume_df.columns = ['ICE Brent Crude', 'NYMEX Light Sweet']

volume_df.plot(kind='bar', stacked=True, ax=ax, color=palette, width=0.8, alpha=1)

ax.set_ylabel('Volume', fontsize=14)
ax.set_xlabel('Date', fontsize=14)
ax.legend(frameon=False, loc='upper left', fontsize=13)
ax.tick_params(axis='x', labelsize=14)

# fix x-tick labels to show month and year and only show every 4th label and .strftime('%Y-%m')
plt.xticks(
    np.arange(0, len(volume_df), 4), 
    volume_df.index[np.arange(0, len(volume_df), 4)].strftime('%Y-%m'), 
    rotation=45
)

fig.savefig('images/crude_oil_volume_stacked.png', bbox_inches='tight', dpi=200)

In [ ]:

df = df_CLc1.copy()

date_picker_from = widgets.DatePicker(
    description='Date from',
    disabled=False,
    value = df.index[0]
)

date_picker_to = widgets.DatePicker(
    description='Date to',
    disabled=False,
    value = df.index[-1]
)

output = widgets.Output()

# Function to handle plotting
def plot_data(b):
    with output:
        clear_output(wait=True)
        start_date = date_picker_from.value
        end_date = date_picker_to.value
        if start_date and end_date:
            filtered_df = df.loc[start_date:end_date]
            plt.figure(figsize=(10,5))
            plt.plot(filtered_df.index, filtered_df['CLOSE'], linestyle='-', lw=1)
            plt.title('Time Series Data between Selected Dates')
            plt.xlabel('Date')
            plt.ylabel('Value')
            plt.grid(True)
            plt.show()

# Button to trigger the plot
plot_button = widgets.Button(description="Plot Data")
plot_button.on_click(plot_data)

# Display the widgets
display(date_picker_from, date_picker_to, plot_button, output)

### Price last 20 years

In [ ]:
df_CLc1_daily = pd.read_csv(rf'{REPO_PATH}data\time_series\CLc1.csv')
df_LCOc1_daily = pd.read_csv(rf'{REPO_PATH}data\time_series\LCOc1.csv')

df_CLc1_daily.index = pd.to_datetime(df_CLc1_daily['Date'])
df_LCOc1_daily.index = pd.to_datetime(df_LCOc1_daily['Date'])

fig, ax = plt.subplots(figsize=(10,4))
palette = [colormap(i) for i in (70, 180)]
df_LCOc1_daily['CLOSE'].plot(label='ICE Brent Crude Oil', ax=ax, lw=0.6, color=palette[0])
df_CLc1_daily['CLOSE'].plot(label='NYMEX Light Sweet Crude Oil', ax=ax, lw=0.6, color=palette[1])
ax.set_ylabel('Price (USD)', fontsize=14)
ax.set_xlabel('Date', fontsize=14)
ax.legend(frameon=False, loc='lower left', fontsize=13)
ax.tick_params(axis='x', labelsize=12)
ax.grid(alpha=0.2)
ax.set_axisbelow(True)

fig.savefig('images/crude_oil_price_daily.png', bbox_inches='tight', dpi=200)
